In [58]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

In [59]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(1)
res =requests.get(url)
soup =BeautifulSoup(res.text)

In [60]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

driver =webdriver.Chrome('file:///C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver')
driver.get(url)

In [61]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [62]:
soup =BeautifulSoup(driver.page_source)
trs =soup.select('.aos-init')
len(trs)

100

In [63]:
tr =trs[0]

In [64]:
rank =tr.select_one('.info_rank').get_text().strip()
rank

'1'

In [65]:
category=tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [66]:
title = tr.select_one('h1 > a').get_text().strip()
title

'BLACKPINK'

In [67]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'8730만'

In [68]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view,video

('305억3350만', '507개')

In [69]:
# 숫자로 바꿔주는 함수
def convert(s):
    s =s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)


In [70]:
convert('333억8260만')

33382600000

In [71]:
lins =[]
for tr in trs:
    rank =int(tr.select_one('.info_rank').get_text().strip())
    category=tr.select_one('.category').get_text().strip()[1:-1]
    title = tr.select_one('h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lins.append({'순위':rank,'카테고리':category,'채널멸':title,'구독자수':subscriber,
                 '조회수':view,'비디오':video})

In [72]:
df1 = pd.DataFrame(lins)
df1

,순위,카테고리,채널멸,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,87300000,30533500000,507
1,2,음악/댄스/가수,BANGTANTV,74500000,20143000000,2188
2,3,음악/댄스/가수,HYBE LABELS,70300000,27323160000,1158
3,4,음악/댄스/가수,SMTOWN,31600000,26794810000,4110
4,5,음악/댄스/가수,JYP Entertainment,26800000,19275070000,1668
...,...,...,...,...,...,...
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450


In [73]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' +str(2)
driver.get(url)
time.sleep(2)
soup =BeautifulSoup(driver.page_source,'html.parser')
trs =soup.select('.aos-init')
len(trs)

100

In [78]:
lins =[]
driver =webdriver.Chrome('file:///C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver')
for page  in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' +str(page)
    driver.get(url)
    time.sleep(2)
    soup =BeautifulSoup(driver.page_source,'html.parser')
    trs =soup.select('.aos-init')

    for tr in trs:
        rank =int(tr.select_one('.info_rank').get_text().strip())
        category=tr.select_one('.category').get_text().strip()[1:-1]
        title = tr.select_one('h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lins.append({'순위':rank,'카테고리':category,'채널명':title,'구독자수':subscriber,
                    '조회수':view,'비디오':video})
driver.close()

In [79]:
df =pd.DataFrame(lins)
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
995,991,회사/오피셜,LG Uplus,730000,1379980000,2473
996,998,BJ/인물/연예인,케이,730000,572670000,3304
997,1001,미분류,백크 Baekk,720000,457320000,1106
998,999,음악/댄스/가수,Crush,720000,395460000,131
999,997,BJ/인물/연예인,bobaepapa - 보배아빠,720000,348440000,393


In [80]:
df.to_csv('data/유튜브 랭킹.csv',index=False)